In [1]:
import dill
import pickle
import pathlib
from pprint import pprint
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from Settings import Settings
from window_based_tagger_config import get_config
from FindFiles import find_files
from DirUtils import dir_exists
from results_procesor import ResultsProcessor, __MICRO_F1__

In [2]:
EMPTY_TAG = "Empty"
ANAPHORA = "Anaphor"

In [3]:
!python -V

Python 3.6.4 :: Anaconda, Inc.


In [4]:
!pip freeze

absl-py==0.3.0
anaconda-client==1.6.11
appnope==0.1.0
argcomplete==1.9.4
asn1crypto==0.24.0
astor==0.7.1
beautifulsoup4==4.6.0
bleach==2.1.2
boto==2.47.0
boto3==1.5.36
botocore==1.8.50
bz2file==0.98
certifi==2018.1.18
cffi==1.11.4
chardet==3.0.4
clyent==1.2.2
costcla==0.5
cryptography==2.1.4
cycler==0.10.0
cymem==1.31.2
cytoolz==0.8.2
decorator==4.2.1
dicecore==1.13
dill==0.2.8.2
docutils==0.14
entrypoints==0.2.3
ftfy==4.4.3
gast==0.2.0
gensim==0.13.4
grpcio==1.14.0
h5py==2.7.0
hdbscan==0.8.12
html5lib==1.0.1
idna==2.6
ipykernel==4.8.2
ipython==6.2.1
ipython-genutils==0.2.0
ipywidgets==7.1.2
jedi==0.11.1
Jinja2==2.10
jmespath==0.9.3
joblib==0.9.4
jsonschema==2.6.0
jupyter==1.0.0
jupyter-client==5.2.2
jupyter-console==5.2.0
jupyter-core==4.4.0
jupyterlab==0.31.8
jupyterlab-launcher==0.10.5
Keras==1.2.2
Keras-Applications==1.0.4
Keras-Preprocessing==1.0.2
Markdown==2.6.11
MarkupSafe==1.0
matplotlib==2.0.0
mistune==0.8.3
murmurhash==0.26.4
nb-anacondacloud==1.4.0
nb-conda==2.2.1
nb-conda-

In [3]:
#DATASET = "CoralBleaching"
DATASET = "SkinCancer"

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + DATASET + "/Thesis_Dataset/training.pl"

# PREDICTIONS FOLDERS
anaphor_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/"

# USE the "-Fixed" ones, these are much worse
#anaphor_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-Anaphora_Tags-Binary/"

tag_predictions_folder = root_folder + "Predictions/Bi-LSTM_fixed/"
#tag_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"
merged_predictions_folder = root_folder + "Predictions/CoRef/MergedTags/"
config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [4]:
# ensure paths exist
pathlib.Path(merged_predictions_folder).mkdir(parents=True, exist_ok=True) 

assert dir_exists(merged_predictions_folder)
print("Valid")

Valid


In [5]:
assert dir_exists(anaphor_predictions_folder)
print("Valid")

Valid


In [6]:
tag_predictions_folder

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM_fixed/'

In [7]:
assert dir_exists(tag_predictions_folder)
print("Valid")

Valid


In [8]:
# get tagged essays
def load_tagged_essays(folder, pattern):
    files = find_files(folder, pattern)
    # multiple runs with different hidden layer sizes?
    if len(files) > 2:        
        for f in files:
            print(f)
    assert len(files) == 2, "Wrong number of tagged files:" + str(len(files))
    for f in files:
        assert "_train_" in f or "_test_" in f, "Wrong files matched"
    train_tagged_fname = [f for file in files if "_train_" in f][0]
    test_tagged_fname = list(set(files).difference([train_tagged_fname]))[0]
    print("Train: {fname}".format(fname=train_tagged_fname))
    print("Test:  {fname}".format(fname=test_tagged_fname))

    # NOTE - is this throws an error, upgrade to dill 2.8.2. Version 2.6 had a bug in it
    with open(train_tagged_fname, "rb") as f:
        tagged_essays_train = dill.load(f)
    with open(test_tagged_fname, "rb") as f:
        tagged_essays_test  = dill.load(f)
    return (tagged_essays_train, tagged_essays_test)

In [58]:
# for optimal vd params - see http://localhost:8888/notebooks/Mongo%20Queries/Query%20Model%20Hyper%20Parameter%20Tuning%20Results-Anaphora%20Tagger.ipynb#
# NOTE that 1 layer is optimal for SC, 2 for CB
#pattern = "essays_.*_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
pattern = "essays_.*_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-1_use_pretrained_embedding-True.dill"
ana_tagged_tr, ana_tagged_test = load_tagged_essays(anaphor_predictions_folder, pattern=pattern)
len(ana_tagged_tr), len(ana_tagged_test)

Train: /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-1_use_pretrained_embedding-True.dill
Test:  /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/essays_test_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-1_use_pretrained_embedding-True.dill


(870, 218)

In [12]:
# load concept code (cc) tagged essays
# see mongo collection - metrics_codes.STORE_RESULTS_SC_TAGGING_VD_RNN_MOST_COMMON_TAG
pattern = "essays_.*_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
cc_tagged_tr, cc_tagged_test = load_tagged_essays(tag_predictions_folder, pattern=pattern)
len(cc_tagged_tr), len(cc_tagged_test)

Train: /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM_fixed/essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill
Test:  /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM_fixed/essays_test_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill


(870, 218)

## Load Raw Essays (Untagged)

In [13]:
# do I need to do this? good for validation below, but not needed otherwise
# with open(training_pickled, "rb+") as f:
#     untagged_essays_train = pickle.load(f)

untagged_essays_train = load_process_essays(**config)

test_config = get_config(test_folder)
untagged_essays_test = load_process_essays(**test_config)

len(untagged_essays_train), len(untagged_essays_test) # 902, 226

870 files found
870 essays processed
218 files found
218 essays processed


(870, 218)

In [14]:
config

{'folder': '/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Training/',
 'include_normal': False,
 'include_vague': True,
 'lower_case': True,
 'min_df': 2,
 'min_sentence_length': 3,
 'remove_infrequent': False,
 'remove_punctuation': False,
 'remove_stop_words': False,
 'replace_nums': True,
 'spelling_correct': True,
 'stem': False,
 'window_size': 9}

## Validate all essay sets are equal and the same

In [15]:
def names_the_same(essay_sets):
    unique_fnames = [] # list of sets of str (fnames)
    for essay_collection in essay_sets:
        names = set()
        for e in essay_collection:
            names.add(e.name)
        unique_fnames.append(names)
    for a in unique_fnames:
        print(len(a))
        for b in unique_fnames:
            assert len(a) == len(b), "lens don't match"
            assert a == b, "don't match"

In [16]:
names_the_same([ana_tagged_tr, cc_tagged_tr, untagged_essays_train])

870
870
870


In [17]:
names_the_same([ana_tagged_test, cc_tagged_test, untagged_essays_test])

218
218
218


In [18]:
def essays_2_hash_map(essays):
    lu = {}
    for e in essays:
        lu[e.name] = e
    return lu

In [24]:
# checks the number of words and sentences are the same for 2 sets of essays
def validate_tagged_essays(essays_a, essays_b, tags_should_match=True):
    # make sure obj is not the same
    assert essays_a != essays_b
    print("Validating", len(essays_a), "essays")
    assert len(essays_a) == len(essays_b), "Lens don't match"
    
    a_hmap = essays_2_hash_map(essays_a)
    b_hmap = essays_2_hash_map(essays_b)
    
    # same essays?
    assert a_hmap.keys() == b_hmap.keys()
    intersect = set(a_hmap.keys()).intersection(b_hmap.keys())
    assert len(intersect) == len(a_hmap.keys())
    assert len(a_hmap.keys()) > 1    
    assert len(a_hmap.keys()) == len(b_hmap.keys())
    
    word_misses = 0
    
    for key, a_essay in a_hmap.items():
        b_essay = b_hmap[key]
        # assert NOT the same obj ref
        assert a_essay != b_essay
        assert len(a_essay.sentences) == len(b_essay.sentences)
        assert len(a_essay.sentences) > 0
        assert len(b_essay.sentences) > 0
        for i in range(len(a_essay.sentences)):
            a_sent = a_essay.sentences[i]
            b_sent = b_essay.sentences[i]
            # the same lists?
            #assert a_sent == b_sent
            assert len(a_sent) == len(b_sent)
            if not len(a_sent) == len(b_sent):
                print(key, "\tsent-ix:", i, "lens", len(a_sent), len(b_sent))
            for wd_ix, (a_wd, a_tags) in enumerate(a_sent):
                b_wd, b_tags = b_sent[wd_ix]
                if a_wd != b_wd:
                    word_misses+=1
                assert a_wd   == b_wd,   \
                    "Words don't match: '{a}' - '{b}', Esssay: {essay} Sent Ix: {i}".format(
                            a=a_wd, b=b_wd, essay=key, i=i)
                
                # SH - Make conditional, as untagged essays contain new anaphora tags
                if tags_should_match:                
                    assert a_tags == b_tags, \
                        "Tags don't match: '{a}' - '{b}', Esssay: {essay} Sent Ix: {i}".format(
                            a=str(a_tags), b=str(b_tags), essay=key, i=i)
                else:
                    intersectn = a_tags.intersection(b_tags)
                    # smaller set should match intersection i.e. be a subset of larger one
                    # will only differ due to new anaphora tags
                    if len(b_tags) <= len(a_tags):
                        assert intersectn == b_tags
                    else:
                        assert intersectn == a_tags
                        
    if word_misses:
        print("Word miss-matches: ", word_misses)
    print("Validation Passed")
    return None

In [30]:
validate_tagged_essays(untagged_essays_train, ana_tagged_tr, tags_should_match=False)
validate_tagged_essays(cc_tagged_tr, ana_tagged_tr)
validate_tagged_essays(untagged_essays_test, ana_tagged_test, tags_should_match=False)
validate_tagged_essays(cc_tagged_test, ana_tagged_test, tags_should_match=False)

Validating 870 essays
Validation Passed
Validating 870 essays
Validation Passed
Validating 218 essays
Validation Passed
Validating 218 essays
Validation Passed


## Merge the Two Sets of Tagged Essays
* The anaphora essays were not tagged with concept codes, and vice versa, so need to merge

In [31]:
def merge_tagged_essays(untagged, tagged_ana, tagged_cc):
    untagged_hmap = essays_2_hash_map(untagged)
    ana_hmap = essays_2_hash_map(tagged_ana)
    cc_hmap = essays_2_hash_map(tagged_cc)
    
    assert ana_hmap.keys() == cc_hmap.keys()
    assert len(ana_hmap.keys()) == len(cc_hmap.keys())
    
    for key, cc_essay in cc_hmap.items():
        ana_essay = ana_hmap[key]
        untag_essay = untagged_hmap[key]
        assert len(ana_essay.sentences) == len(cc_essay.sentences)
        untag_essay.ana_tagged_sentences  = ana_essay.pred_tagged_sentences
        untag_essay.pred_tagged_sentences =  cc_essay.pred_tagged_sentences
        for i in range(len(ana_essay.sentences)):
            ana_sent = ana_essay.sentences[i]
            ana_ptags = ana_essay.pred_tagged_sentences[i]
            cc_sent = ana_essay.sentences[i]
            cc_ptags = cc_essay.pred_tagged_sentences[i]
            assert len(ana_sent) == len(cc_sent)
            assert len(cc_sent) == len(cc_ptags)
    return untagged

# Train
merged_essays_tr   = merge_tagged_essays(
    untagged=untagged_essays_train, 
    tagged_ana=ana_tagged_tr,   
    tagged_cc=cc_tagged_tr)

# Test
merged_essays_test = merge_tagged_essays(
    untagged=untagged_essays_test,
    tagged_ana=ana_tagged_test, 
    tagged_cc=cc_tagged_test)
len(merged_essays_tr), len(merged_essays_test)

(870, 218)

## Re-Compute Accuracy Metrics

In [32]:
e = untagged_essays_train[0]
len(e.sentences),len(e.pred_tagged_sentences),len(e.ana_tagged_sentences)

(19, 19, 19)

In [33]:
e = untagged_essays_test[0]
len(e.sentences),len(e.pred_tagged_sentences),len(e.ana_tagged_sentences)

(1, 1, 1)

In [34]:
def norm_tally(tally):
    total = sum(tally.values())
    norm_tally = {}
    for tag, freq in tally.items():
        norm_tally[tag] = freq/total
    return norm_tally

### Get labels


In [35]:
tally = defaultdict(int)
for e in untagged_essays_train:
    for sent in e.sentences:
        for wd, tags in sent:
            for t in tags:
                tally[t] +=1

lst_all_tags = list(tally.keys())
regular_tags = sorted(set((t for t in lst_all_tags if t[0].isdigit())), key = lambda s: int(s.replace('b','')))
assert EMPTY_TAG not in regular_tags, "Empty tag in list of regular tags"
regular_tags

['1', '2', '3', '4', '5', '6', '11', '12', '50']

### Compare Label Distributions

In [36]:
cc_tally = dict([k,v] for k,v in tally.items() if k in set(regular_tags))
sorted(norm_tally(cc_tally).items(), key = lambda tpl: tpl[0])

[('1', 0.14999496661185865),
 ('11', 0.01171101640884534),
 ('12', 0.010838562464346835),
 ('2', 0.1533505587060837),
 ('3', 0.09439280561054998),
 ('4', 0.07409147344048858),
 ('5', 0.13341834166638702),
 ('50', 0.2954263279755713),
 ('6', 0.07677594711586859)]

In [37]:
ptag_tally = defaultdict(int)
for e in merged_essays_tr:   
    for sent in e.pred_tagged_sentences:
        for tag in sent:
            ptag_tally[tag] +=1
            
norm_ptag_tally = norm_tally(dict([(k,v) for k,v in ptag_tally.items() if k != EMPTY_TAG]))
sorted(norm_ptag_tally.items(), key = lambda tpl: tpl[0])

[('1', 0.15222876366694701),
 ('11', 0.00963694981777404),
 ('12', 0.009882253994953743),
 ('2', 0.1559784132324082),
 ('3', 0.09668488926268573),
 ('4', 0.0787426408746846),
 ('5', 0.13589851415755536),
 ('50', 0.2989206616204093),
 ('6', 0.062026913372582)]

In [38]:
list(ptag_tally.keys())

['Empty', '50', '1', '2', '12', '3', '5', '4', '6', '11']

## Validate Metrics on Concept Codes

### Train

In [39]:
from results_procesor import metrics_to_df
import numpy as np
METRICS_COLS = ["code","f1_score", "precision", "recall", "accuracy", "data_points"]

reg_tr_metrics = ResultsProcessor.compute_mean_metrics_from_tagged_essays(cc_tagged_tr, regular_tags)
m_df = metrics_to_df(reg_tr_metrics)
m_df = m_df[np.isin(m_df["code"], regular_tags + [__MICRO_F1__])][METRICS_COLS]
m_df.sort_values("code")

,code,f1_score,precision,recall,accuracy,data_points
0,1,0.815067,0.826888,0.803579,0.988795,145471.0
7,11,0.746795,0.847273,0.667622,0.998914,145471.0
3,12,0.641322,0.687943,0.600619,0.998508,145471.0
8,2,0.791043,0.801618,0.780744,0.987042,145471.0
6,3,0.808686,0.816600,0.800924,0.992672,145471.0
5,4,0.777553,0.770806,0.784420,0.993188,145471.0
4,5,0.854596,0.865395,0.844064,0.992150,145471.0
1,50,0.868005,0.881946,0.854498,0.984272,145471.0
2,6,0.758502,0.869492,0.672640,0.993263,145471.0
13,MICRO_F1,0.822463,0.840692,0.805007,0.992089,1309239.0


### Test

In [40]:
reg_test_metrics = ResultsProcessor.compute_mean_metrics_from_tagged_essays(cc_tagged_test, regular_tags)
m_df = metrics_to_df(reg_test_metrics)
m_df = m_df[np.isin(m_df["code"], regular_tags + [__MICRO_F1__])][METRICS_COLS]
m_df.sort_values("code")

,code,f1_score,precision,recall,accuracy,data_points
0,1,0.836207,0.867039,0.807492,0.991413,35402.0
7,11,0.641509,0.850000,0.515152,0.998927,35402.0
3,12,0.646707,0.683544,0.613636,0.998333,35402.0
8,2,0.865702,0.856851,0.874739,0.992656,35402.0
6,3,0.843869,0.849693,0.838124,0.994209,35402.0
5,4,0.772031,0.787109,0.757519,0.993277,35402.0
4,5,0.863286,0.827696,0.902074,0.992995,35402.0
1,50,0.875325,0.888142,0.862873,0.985114,35402.0
2,6,0.746060,0.800752,0.698361,0.991808,35402.0
13,MICRO_F1,0.840573,0.851273,0.830139,0.993192,318618.0


## Validate Metrics on Ana Codes

In [41]:
ana_tags = [ANAPHORA]

### Train

In [42]:
reg_tr_metrics = ResultsProcessor.compute_mean_metrics_from_tagged_essays(ana_tagged_tr, ana_tags)
m_df = metrics_to_df(reg_tr_metrics)
m_df = m_df[np.isin(m_df["code"], regular_tags + [__MICRO_F1__])][METRICS_COLS]
m_df.sort_values("code")

,code,f1_score,precision,recall,accuracy,data_points
5,MICRO_F1,0.350575,0.552036,0.256842,0.996893,145471.0


### Test

In [43]:
reg_test_metrics = ResultsProcessor.compute_mean_metrics_from_tagged_essays(ana_tagged_test, ana_tags)
m_df = metrics_to_df(reg_test_metrics)
m_df = m_df[np.isin(m_df["code"], regular_tags + [__MICRO_F1__])][METRICS_COLS]
m_df.sort_values("code")

,code,f1_score,precision,recall,accuracy,data_points
5,MICRO_F1,0.47343,0.505155,0.445455,0.996921,35402.0


# Persist Merged Essays

In [44]:
merged_predictions_folder

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/CoRef/MergedTags/'

In [45]:
with open("{folder}merged_essays_train.dill".format(folder=merged_predictions_folder), "wb+") as f:
    dill.dump(merged_essays_tr, f)

In [46]:
with open("{folder}merged_essays_test.dill".format(folder=merged_predictions_folder), "wb+") as f:
    dill.dump(merged_essays_test, f)

## Test Metrics for All Tagged Essays

In [84]:
import pandas as pd

def get_params_from_name(fname):
    prms = {}
    fname = f.split("/")[-1]
    params = fname.replace(".dill","")
    current = []
    for p in params.split("_")[2:]: # skip essays_test
        if "-" in p:
            end, val = p.split("-")
            current.append(end)
            prms[" ".join(current)] = val
            current = []
        else:
            current.append(p)
    return prms

cols = ['f1_score', 'accuracy', 'precision','recall', 'bi directional', 'data_points', 
          'hidden size', 'merge mode', 'num rnns', 'num_codes', 'use pretrained embedding']

In [99]:
df.sort_values("f1_score", ascending=False)

,accuracy,bi directional,data_points,f1_score,hidden size,merge mode,num rnns,num_codes,precision,recall,use pretrained embedding
3,0.997062,True,35402.0,0.514019,256,sum,2,110.0,0.528846,0.500000,True
1,0.997232,True,35402.0,0.494845,128,sum,2,110.0,0.571429,0.436364,True
2,0.996921,True,35402.0,0.473430,256,sum,1,110.0,0.505155,0.445455,True
0,0.997204,True,35402.0,0.464865,128,sum,1,110.0,0.573333,0.390909,True
4,0.996921,True,35402.0,0.423280,64,sum,1,110.0,0.506329,0.363636,True
5,0.997062,True,35402.0,0.350000,64,sum,2,110.0,0.560000,0.254545,True


In [95]:
pattern = "essays_train_bi_directional-True_hidden_size.*.dill"
files = find_files(anaphor_predictions_folder, pattern)

rows = []
for f in files:
    params = get_params_from_name(f)
    with open(f, "rb+") as fin:
        essays = dill.load(fin)
    metrics = ResultsProcessor.compute_mean_metrics_from_tagged_essays(essays, ana_tags)
    params.update(metrics["MICRO_F1"])
    rows.append(params)

df = pd.DataFrame(rows)
df.sort_values("f1_score", ascending=False)[cols]

,f1_score,accuracy,precision,recall,bi directional,data_points,hidden size,merge mode,num rnns,num_codes,use pretrained embedding
2,0.350575,0.996893,0.552036,0.256842,True,145471.0,256,sum,1,475.0,True
3,0.338139,0.996529,0.447917,0.271579,True,145471.0,256,sum,2,475.0,True
1,0.319508,0.996955,0.590909,0.218947,True,145471.0,128,sum,2,475.0,True
0,0.197917,0.996824,0.564356,0.120000,True,145471.0,128,sum,1,475.0,True
5,0.143357,0.996632,0.422680,0.086316,True,145471.0,64,sum,2,475.0,True
4,0.000000,0.996735,0.000000,0.000000,True,145471.0,64,sum,1,475.0,True


In [96]:
pattern = "essays_test_bi_directional-True_hidden_size.*.dill"
files = find_files(anaphor_predictions_folder, pattern)

rows = []
for f in files:
    params = get_params_from_name(f)
    with open(f, "rb+") as fin:
        essays = dill.load(fin)
    metrics = ResultsProcessor.compute_mean_metrics_from_tagged_essays(essays, ana_tags)
    params.update(metrics["MICRO_F1"])
    rows.append(params)

df = pd.DataFrame(rows)
df.sort_values("f1_score", ascending=False)[cols]

,f1_score,accuracy,precision,recall,bi directional,data_points,hidden size,merge mode,num rnns,num_codes,use pretrained embedding
3,0.514019,0.997062,0.528846,0.500000,True,35402.0,256,sum,2,110.0,True
1,0.494845,0.997232,0.571429,0.436364,True,35402.0,128,sum,2,110.0,True
2,0.473430,0.996921,0.505155,0.445455,True,35402.0,256,sum,1,110.0,True
0,0.464865,0.997204,0.573333,0.390909,True,35402.0,128,sum,1,110.0,True
4,0.423280,0.996921,0.506329,0.363636,True,35402.0,64,sum,1,110.0,True
5,0.350000,0.997062,0.560000,0.254545,True,35402.0,64,sum,2,110.0,True
